# Импорт либ

In [1]:
#!pip install transformers

In [2]:
import os
import math
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
from torch.optim import AdamW
from torch.optim.lr_scheduler import LambdaLR
from functools import partial
from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    TrainingArguments,
    Trainer,
    get_linear_schedule_with_warmup,
    BertConfig
)

# Отключаем WandB и логи от Transformers
os.environ["WANDB_DISABLED"] = "true"
os.environ["TRANSFORMERS_NO_WANDB"] = "true"

## Загрузка данных

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
file_path = "/content/drive/MyDrive/IMDB Dataset.csv"
df = pd.read_csv(file_path)
df['sentiment'] = df['sentiment'].map({'negative': 0, 'positive': 1})

In [5]:
df = df.head(5000) # обрезаю выборку чтобы не терять время на обучении

In [6]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1
...,...,...
4995,An interesting slasher film with multiple susp...,0
4996,i watched this series when it first came out i...,1
4997,Once again Jet Li brings his charismatic prese...,1
4998,"I rented this movie, after hearing Chris Gore ...",0


# Предобработка аналогично дообучению берта

In [7]:
train_val_df, test_df = train_test_split(df, test_size=0.3, random_state=42) #для сопоставимости резов внутри команды фиксируем 42 рандом сид и отрезаем 30 процентов на тест
train_df, val_df = train_test_split(train_val_df, test_size=0.2, random_state=42) # теперь на трейне отрезаю тренировочный кусок и валидационный, пропорция 80/20
print("Train size:", len(train_df))
print("Validation size:", len(val_df))
print("Test size:", len(test_df))

Train size: 2800
Validation size: 700
Test size: 1500


## Токенайзер

In [8]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

## Датасеты и даталоадеры

In [9]:
class IMDBDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        # принимаю фрейм с данными, токенизатор и максимальную длину последовательности
        self.data = dataframe.reset_index(drop=True)
        # сейвлю токенизатор для преобразования текста в токены
        self.tokenizer = tokenizer
        # запоминаю максимальную длину последовательности
        self.max_length = max_length

    def __len__(self):
        #  __len__ возвращает общее число примеров в фрейме
        return len(self.data)

    def __getitem__(self, idx):
        # __getitem__ позволяет получить конкретный пример по индексу

        # получаю текст отзыва по индексу
        review = self.data.iloc[idx]['review']
        # Получаю метку для этого отзыва - бинарнй таргет
        label = self.data.iloc[idx]['sentiment']

        # Токенизирую текст отзыва с помощью заданного выше предобученного токенизатора
        # padding паддинг до фиксированной длины
        # truncation : если текст длиннее макс длины, то он укорачивается
        # max_length: максимальная длина последовательности
        # return_tensors="pt": результат как торч тензор
        encoding = self.tokenizer(
            review,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        # тк возвращаемые торч тензоры имеют размерность с дополнительным измерением,
        # я использую .squeeze(0), чтобы убрать это измерение и получить одномерные тензоры
        item = {key: tensor.squeeze(0) for key, tensor in encoding.items()}

        # + метка к моему словарю"labels", потому что сначала использовал импортнутый с библиотеки датасет и там
        # таргет назывался labels, поэтому немного костылей
        item['labels'] = torch.tensor(label, dtype=torch.long)
        return item

In [10]:
train_dataset = IMDBDataset(train_df, tokenizer)
val_dataset = IMDBDataset(val_df, tokenizer)
test_dataset = IMDBDataset(test_df, tokenizer)

In [11]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

In [12]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

# Random Search Strategy

### В этом классе реализован метод Random Search NAS — самый простой и базовый подход к автоматическому поиску архитектур.

Идея этого подхода, заключается в следующем:

Определяется пространство поиска архитектур (Search Space) — все допустимые конфигурации модели.
В моей реализации - это подмножество слоёв энкодера BERT: из 12 слоёв случайным образом выбирается комбинация от 4 до 12.

Применяется стратегия случайного выбора — архитектуры выбираются случайным образом из пространства поиска, без градиентных шагов или обучения мета-моделей. Это делает интуитивно понятным и простым.

Вводится стратегия оценки производительности (Performance Estimation Strategy) — каждая случайная архитектура обучается и оценивается по прокси-метрикам. Здесь:

- используется accuracy на валидации,
- количество параметров модели (total_params),
- кастомная метрика, которая балансирует точность и компактность модели (трейд-офф между аккураси и весом)
- время обучения

Класс реализует многократную итерацию случайного выбора архитектур, обучение каждой из них на одной эпохе, и сохранение результатов. Это позволяет выявить простые, но эффективные архитектуры без применения сложных стратегий.

In [14]:
class RandomSearchNAS:
    """
    Реализую Random Search NAS для подбора наилучшей архитектуры берта для нашей задачи

    Здесь реализованы 3 компоненты NAS:
    1. Search Space — задаю как все возможные комбинации слоев
    2. Search Strategy — случайно выбирается одна из всего множества стратегий
    3. Performance Estimation Strategy — обучение и оценка по аккураси, времени обучения, числу параметров и моей метрике скора,
    которая зависит от аккураси и количества параметров
    """

    def __init__(self, train_dataset, val_dataset, num_trials=10,
                 dropout_rate=0.1, classifier_hidden=768,
                 num_layers_in_head=1, activation="gelu",
                 alpha=0.7, beta=0.3, max_params=110_000_000):
        # Инициализация параметров и  входных данных
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.num_trials = num_trials
        self.used_configs = set()
        self.results = []

        # Настройки классификатора
        self.dropout_rate = dropout_rate
        self.classifier_hidden = classifier_hidden
        self.num_layers_in_head = num_layers_in_head
        self.activation = activation

        # Мои кастомные метрики
        self.alpha = alpha
        self.beta = beta
        self.max_params = max_params

    def sample_architecture(self, trial_num=None, max_retries=50):
        """
        Случайный выбор архитектуры:
        Выбираю подмножество слоев энкодера - от 4 до 12
        """
        retries = 0
        while retries < max_retries:
            num_layers = 12
            k = random.randint(4, 12)
            layer_indices = tuple(sorted(random.sample(range(num_layers), k)))

            config_hash = (layer_indices,)

            if config_hash not in self.used_configs:
                self.used_configs.add(config_hash)
                arch = {"layer_indices": list(layer_indices)}
                if trial_num is not None:
                    print(f"Случайно-выбранная архитекура с параметрами {trial_num}: {arch}")
                return arch

            retries += 1

        raise ValueError("Не удалось сгенерировать уникальную архитектуру") # добавил тк вылетала ошибка с одной и той же моделью

    def compute_metrics_wrapper(self, eval_pred, total_params):
        """
        Обёртка нужна для Trainer — она считает только eval_accuracy
        """
        logits, labels = eval_pred
        preds = np.argmax(logits, axis=-1)
        acc = accuracy_score(labels, preds)
        return {"eval_accuracy": acc}

    def compute_metrics(self, eval_pred, total_params, train_time):
        """
        Perf estimation strategy: accuracy и кастомной метрики, самый простой вариант на одной эпохе
        учитывает точность и количество параметров (можно играть параметрами альфа и бета)
        """
        logits, labels = eval_pred
        preds = np.argmax(logits, axis=-1)
        acc = accuracy_score(labels, preds)

        # Нормализация параметров, уменьшаю вклад больших моделей
        norm_param_score = 1 - (math.log(total_params + 1) / math.log(self.max_params))
        custom_score = self.alpha * acc + self.beta * norm_param_score

        return {
            "eval_accuracy": acc,
            "total_params": total_params,
            "train_time": train_time,
            "custom_score": custom_score
        }

    def evaluate_architecture(self, arch_config):
        """
        Строю и обучаю модель с выбранной архитектурой
        Обрезаю encoder до нужных слоёв, обучаю 1 эпоху и считаю метрики
        """
        # Загружаю берт и токенайзер
        model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
        tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

        # Оставляю только выбранные слои энкодера
        selected_layers = arch_config["layer_indices"]
        model.bert.encoder.layer = nn.ModuleList([model.bert.encoder.layer[i] for i in selected_layers])
        model.config.num_hidden_layers = len(selected_layers)

        # Создаю классификатор
        in_features = model.classifier.in_features
        activation_fn = nn.ReLU() if self.activation == "relu" else nn.GELU()

        if self.num_layers_in_head == 1:
            classifier = nn.Sequential(
                nn.Dropout(self.dropout_rate),
                nn.Linear(in_features, 2)
            )
        else:
            classifier = nn.Sequential(
                nn.Dropout(self.dropout_rate),
                nn.Linear(in_features, self.classifier_hidden),
                activation_fn,
                nn.Dropout(self.dropout_rate),
                nn.Linear(self.classifier_hidden, 2)
            )

        model.classifier = classifier
        total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

        # Настройки обучения
        training_args = TrainingArguments(
            output_dir="./results",
            evaluation_strategy="steps",
            eval_steps=1000,
            logging_strategy="steps",
            logging_steps=1000,
            save_strategy="steps",
            save_steps=1000,
            load_best_model_at_end=True,
            metric_for_best_model="eval_accuracy",
            greater_is_better=True,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=16,
            num_train_epochs=1,
            learning_rate=2e-5,
            logging_dir="./logs",
            report_to="none",
            disable_tqdm=False
        )

        # Trainer с логированием accuracy во время обучения
        from functools import partial
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=self.train_dataset,
            eval_dataset=self.val_dataset,
            tokenizer=tokenizer,
            compute_metrics=partial(self.compute_metrics_wrapper, total_params=total_params)
        )

        # время обучения
        start_time = time.time()
        trainer.train()
        train_time = time.time() - start_time

        # итоговая метрика (полная), считается отдельно
        predictions = trainer.predict(self.val_dataset)
        logits = predictions.predictions
        labels = predictions.label_ids

        return self.compute_metrics((logits, labels), total_params=total_params, train_time=train_time)

    def run(self):
        """
        Запуск поиска архитектуры, просто n раз рандомно выбираем комбинацию слоев
        """
        print(f"Рандомный поиск| Итераций: {self.num_trials}\n")
        for i in tqdm(range(1, self.num_trials + 1), desc="Random Search"):
            print(f"\n Попытка номер {i}/{self.num_trials}")
            try:
                #Search Strategy
                arch = self.sample_architecture(trial_num=i)

                #Performance Estimation Strategy
                metrics = self.evaluate_architecture(arch)

                arch.update(metrics)
                arch["trial"] = i
                self.results.append(arch)

                print(f"Accuracy = {metrics['eval_accuracy']:.4f}, "
                      f"Params = {metrics['total_params']:,}, "
                      f"Time = {metrics['train_time']:.2f}s, "
                      f"Custom Score = {metrics['custom_score']:.4f}")
            except Exception as e:
                print(f"Ошибка в запуске #{i}: {e}")

        # Топ 5 архитектур по кастомному скору
        results_df = pd.DataFrame(self.results).sort_values(by="custom_score", ascending=False)
        print("\n Топ-5 архитектур \n")
        display(results_df.head())
        return results_df

## Запускаем поиск, всего сделал 3 выбора

In [15]:
nas = RandomSearchNAS(
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    num_trials=3,
    alpha=0.3,
    beta=0.7
)

results_df = nas.run()

Рандомный поиск| Итераций: 3



Random Search:   0%|          | 0/3 [00:00<?, ?it/s]


 Попытка номер 1/3
Случайно-выбранная архитекура с параметрами 1: {'layer_indices': [0, 2, 3, 4, 5, 6, 7, 8, 9, 10]}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-14-5aa235dd94c3>:145: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss


...

Accuracy = 0.8329, Params = 59,868,674, Time = 123.11s, Custom Score = 0.2729

 Попытка номер 3/3
Случайно-выбранная архитекура с параметрами 3: {'layer_indices': [0, 1, 6, 8, 10]}


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-14-5aa235dd94c3>:145: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss


Accuracy = 0.8429, Params = 59,868,674, Time = 124.65s, Custom Score = 0.2759

 Топ-5 архитектур 



,layer_indices,eval_accuracy,total_params,train_time,custom_score,trial
0,"[0, 2, 3, 4, 5, 6, 7, 8, 9, 10]",0.915714,95308034,227.117790,0.280134,1
2,"[0, 1, 6, 8, 10]",0.842857,59868674,124.648224,0.275855,3
1,"[0, 2, 3, 4, 9]",0.832857,59868674,123.112717,0.272855,2


# Возьму модель со слоями [0, 1, 6, 8, 10]